# 11 - Fill Mask (Relleno de espacios)

<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/fill_mask.webp" style="width:400px;"/>

<h1>Tabla de Contenidos<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---Modelos-Fill-Mask" data-toc-modified-id="1---Modelos-Fill-Mask-1">1 - Modelos Fill Mask</a></span></li><li><span><a href="#2---Pipeline-de-Transformers-para-rellenar" data-toc-modified-id="2---Pipeline-de-Transformers-para-rellenar-2">2 - Pipeline de Transformers para rellenar</a></span></li><li><span><a href="#3---Usando-el-modelo-de-rellenado" data-toc-modified-id="3---Usando-el-modelo-de-rellenado-3">3 - Usando el modelo de rellenado</a></span><ul class="toc-item"><li><span><a href="#3.1-Tokenizador" data-toc-modified-id="3.1-Tokenizador-3.1">3.1 Tokenizador</a></span></li><li><span><a href="#3.2---Modelo-de-rellenado-(fill-mask)" data-toc-modified-id="3.2---Modelo-de-rellenado-(fill-mask)-3.2">3.2 - Modelo de rellenado (fill mask)</a></span></li></ul></li><li><span><a href="#4---Otro-modelo-de-relleno" data-toc-modified-id="4---Otro-modelo-de-relleno-4">4 - Otro modelo de relleno</a></span></li></ul></div>

## 1 - Modelos Fill Mask

Los modelos Fill Mask, o de rellenado de espacios, son un tipo de modelo de NLP que se utiliza principalmente para predecir palabras faltantes en una oración. Estos modelos son entrenados para entender el contexto de una oración y seleccionar la palabra que mejor completa el espacio en blanco designado por una máscara.

**Funcionamiento de los modelos Fill Mask**:

1. Preentrenamiento: Durante la fase de preentrenamiento, estos modelos son expuestos a grandes cantidades de texto. Un porcentaje de las palabras en el texto se oculta (se "enmascara") y el modelo debe predecir la palabra correcta para cada máscara basándose en el contexto proporcionado por las palabras restantes.


2. Entrenamiento de Tareas Específicas: Después del preentrenamiento, los modelos pueden ser ajustados para tareas específicas que requieran comprensión del lenguaje, aunque el uso principal sigue siendo la predicción de texto.



**Ejemplos de modelos Fill Mask**:


1. BERT (Bidirectional Encoder Representations from Transformers): Uno de los modelos más conocidos que utiliza la técnica de máscara. BERT introduce tokens de máscara en el texto y entrena el modelo para predecir la palabra original en la posición de la máscara.


2. RoBERTa (Robustly Optimized BERT Approach): Una variante de BERT que ha sido optimizada y ajustada para mejorar el rendimiento en tareas de predicción de palabras enmascaradas.



**Aplicaciones de los modelos Fill Mask**:


+ Mejora de Sistemas de Recomendación de Palabras: Utilizados en editores de texto y software de procesamiento de palabras para ofrecer sugerencias de palabras o para completar automáticamente frases.


+ Entrenamiento de Modelos de Lenguaje: Ayuda a mejorar la capacidad de los modelos para entender y generar lenguaje natural.


+ Análisis de Sentimiento y Extracción de Información: Al entender mejor el contexto de las palabras dentro de las frases, estos modelos pueden ayudar en la extracción de información relevante y en la evaluación de sentimientos en textos.


## 2 - Pipeline de Transformers para rellenar

Vamos a usar un modelo [BERT](https://huggingface.co/google-bert/bert-base-multilingual-cased), creado por Google, para realizar la tarea de rellenado de espacios vacíos usando el pipeline de transformers. Tiene un aproximado de 720Mb. Está preentrenado con 104 idiomas usando las Wikipedia. Fue presentado por primera vez en este [paper](https://arxiv.org/abs/1810.04805). 


El modelado de lenguaje enmascarado (MLM) toma una oración, el modelo enmascara aleatoriamente el 15% de las palabras en la entrada, luego ejecuta la oración enmascarada completa a través del modelo y tiene que predecir las palabras enmascaradas. Esto es diferente de las redes neuronales recurrentes tradicionales (RNN) que generalmente ven las palabras una tras otra, o de modelos autorregresivos como GPT que internamente enmascaran los tokens futuros. Esto permite que el modelo aprenda una representación bidireccional de la oración.

Para la predicción de la siguiente oración, estos modelos concatenan dos oraciones enmascaradas como entradas durante el preentrenamiento. A veces corresponden a oraciones que estaban una al lado de la otra en el texto original, a veces no. Luego, el modelo tiene que predecir si las dos oraciones seguían una a la otra o no.
De esta manera, el modelo aprende una representación interna de los idiomas en el conjunto de entrenamiento que luego puede ser utilizada para extraer características útiles para tareas posteriores, si tenemos un conjunto de datos de oraciones etiquetadas, por ejemplo, podemos entrenar un clasificador estándar usando las características producidas por el modelo BERT como entradas.


Para usar el modelo y hacer que rellene una frase incompleta, debemos usar el token especial `[MASK]` para indicarle que posición ocupa la palabra que tiene que generar.

In [1]:
from transformers import pipeline

In [2]:
tarea = 'fill-mask'

modelo = 'google-bert/bert-base-multilingual-cased'

In [7]:
fill_pipe = pipeline(task=tarea, model=modelo, device='cpu')

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
frase = 'La capital de España es [MASK].'

In [11]:
fill_pipe(frase)

[{'score': 0.6894896030426025,
  'token': 11727,
  'token_str': 'Madrid',
  'sequence': 'La capital de España es Madrid.'},
 {'score': 0.039343804121017456,
  'token': 24174,
  'token_str': 'Zaragoza',
  'sequence': 'La capital de España es Zaragoza.'},
 {'score': 0.020669730380177498,
  'token': 40603,
  'token_str': 'Málaga',
  'sequence': 'La capital de España es Málaga.'},
 {'score': 0.020436637103557587,
  'token': 35527,
  'token_str': 'Valladolid',
  'sequence': 'La capital de España es Valladolid.'},
 {'score': 0.01399324368685484,
  'token': 41774,
  'token_str': 'Santander',
  'sequence': 'La capital de España es Santander.'}]

Vemos que salida del pipeline nos devuelve una lista de diccionarios. Cada uno de ellos tiene cuatro keys:


+ score: Probabilidad de respuesta correcta. Es un float.


+ token: Id del token predicho. Es un número entero.


+ token_str: Token predicho en formato string.


+ sequence: Frase completa rellenada con el token predicho en formato string.


Los diccionarios dentro de la lista están ordenados según la probabilidad de la respuesta correcta.

## 3 - Usando el modelo de rellenado

Usemos el modelo fuera del pipeline. Como siempre, necesitamos el tokenizador y el modelo preentrenado:

In [12]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

### 3.1 Tokenizador

In [13]:
tokenizador = AutoTokenizer.from_pretrained(modelo)

In [14]:
tokenizador

BertTokenizerFast(name_or_path='google-bert/bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

Ya vimos anteriormente la descripción de este objeto tokenizador `BertTokenizerFast`. Los tokens especiales son: [PAD], [UNK], [CLS], [SEP] y [MASK].

In [15]:
vector = tokenizador(frase, return_tensors='pt')

In [16]:
vector

{'input_ids': tensor([[  101, 10159, 12185, 10104, 12413, 10196,   103,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [17]:
vector.tokens()

['[CLS]', 'La', 'capital', 'de', 'España', 'es', '[MASK]', '.', '[SEP]']

### 3.2 - Modelo de rellenado (fill mask)

In [18]:
modelo_rellenado = AutoModelForMaskedLM.from_pretrained(modelo)

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
modelo_rellenado

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

**Componentes de BertForMaskedLM**:


1. **BertModel**: Es el modelo base que contiene la arquitectura fundamental de BERT, incluyendo embeddings y capas de encoder.

+ Embeddings: Procesa las entradas de texto en vectores antes de pasar por las capas del encoder.
    + word_embeddings: Transforma los tokens de entrada en vectores.
    + position_embeddings: Agrega información posicional a los embeddings de los tokens para mantener el orden de la secuencia.
    + token_type_embeddings: Utilizado para diferenciar entre tipos de tokens (por ejemplo, en tareas que involucran dos secuencias diferentes como preguntas y respuestas).
    + LayerNorm y Dropout: Son técnicas de normalización y regularización, respectivamente, para mejorar el entrenamiento y evitar el sobreajuste.



2. **BertEncoder**: Consiste en múltiples capas de BertLayer, cada una de las cuales contiene componentes para realizar la atención y la transformación de los datos:

+ BertAttention: Se encarga de la atención multi-cabeza para capturar diferentes aspectos de la información en diferentes posiciones de los datos.
    + BertSelfAttention: Realiza cálculos de atención sobre los embeddings, usando tres transformaciones lineales para crear consultas, claves y valores.
+ BertIntermediate: Aplica una transformación lineal para expandir las dimensiones de los datos antes de aplicar una función de activación no lineal (GELU).
+ BertOutput: Combina la salida de la atención y la capa intermedia, y luego la proyecta de nuevo al tamaño original de los embeddings.


3. **cls**: Específico para tareas MLM, contiene componentes para la predicción de tokens:

+ BertOnlyMLMHead: Cabeza del modelo diseñada específicamente para predecir los tokens ocultos en una tarea MLM.
    + BertLMPredictionHead: Transforma la salida del encoder antes de predecir el token original.
        + BertPredictionHeadTransform: Aplica transformaciones adicionales antes de la predicción final.
        + decoder: Transforma la salida del modelo a un espacio de tamaño igual al vocabulario para predecir el token enmascarado.



**Funcionamiento general**:
El proceso comienza con la entrada de texto, que se tokeniza y se convierte en embeddings. Estos embeddings pasan a través de múltiples capas de atención y transformaciones lineales dentro del encoder. La salida del encoder se procesa luego para predecir tokens enmascarados, que es el objetivo principal del modelo BertForMaskedLM.


In [20]:
respuesta = modelo_rellenado(**vector)

In [23]:
logits = respuesta.logits

logits.shape

torch.Size([1, 9, 119547])

In [28]:
vector.input_ids == tokenizador.mask_token_id

tensor([[False, False, False, False, False, False,  True, False, False]])

In [30]:
mask_index = (vector.input_ids == tokenizador.mask_token_id)[0].nonzero()[0]

mask_index

tensor([6])

In [35]:
logits[0, mask_index].argmax(axis=-1)

tensor([11727])

In [38]:
resp = tokenizador.decode(logits[0, mask_index].argmax(axis=-1))

resp

'Madrid'

In [39]:
frase.replace('[MASK]', resp)

'La capital de España es Madrid.'

## 4 - Otro modelo de relleno

Probemos otro modelo, [XLM-RoBERTa](https://huggingface.co/FacebookAI/xlm-roberta-large) desarrollado por Facebook. Este modelo es más grande que el anterior, pesa aproximadamente 2.25Gb. Vamos a usarlo a través del pipeline. Hay que decir que este modelo usa como token especial `<mask>`, distinto del anterior. El modelo XLM-RoBERTa está preentrenado en 2.5TB de datos de CommonCrawl filtrados que contienen 100 idiomas. Fue presentado en el siguiente [paper](https://arxiv.org/abs/1911.02116).


In [40]:
modelo = 'FacebookAI/xlm-roberta-large'

In [41]:
pipe = pipeline(task=tarea, model=modelo)

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [46]:
frase = 'La capital de España es <mask>'

In [47]:
pipe(frase)

[{'score': 0.7915661931037903,
  'token': 8884,
  'token_str': 'Madrid',
  'sequence': 'La capital de España es Madrid'},
 {'score': 0.1014159694314003,
  'token': 5755,
  'token_str': 'Barcelona',
  'sequence': 'La capital de España es Barcelona'},
 {'score': 0.05767650902271271,
  'token': 12,
  'token_str': ':',
  'sequence': 'La capital de España es:'},
 {'score': 0.009059983305633068,
  'token': 73015,
  'token_str': 'Valencia',
  'sequence': 'La capital de España es Valencia'},
 {'score': 0.00565499160438776,
  'token': 27,
  'token_str': '...',
  'sequence': 'La capital de España es...'}]